In [1]:
import h3 as h3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy import stats
from pyhive import presto
from keplergl import KeplerGl
from datetime import datetime, timedelta

import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 1000)

## Connection

In [3]:
## Connection
connection = presto.connect(
        host='presto-gateway.serving.data.production.internal',
        port=80,
        protocol='http',
        catalog='hive',
        username='manoj.ravirajan@rapido.bike'
)

## Dataset

In [4]:
## Generate date range

# start_date = datetime(2023, 7, 10)  
# end_date = datetime(2023, 7, 16)
startdate = '20231001'
enddate = '20231005'
city = 'Lucknow'
service = 'Link'
# date_range = [start_date + timedelta(days=x) for x in range((end_date - start_date).days + 1)]

In [5]:
## datasets.service_mapping

service_mapping = f"""
        SELECT 
            city_display_name AS city,
            service_level AS service_name,
            service_detail_id,
            city_id,
            service_id
        FROM 
            datasets.service_mapping
        WHERE 
            city_display_name = '{city}'
            AND service_level IN ('Link', 'Bike Lite')
"""

df_service_mapping = pd.read_sql(service_mapping, connection)
# service_detail_id = df_service_mapping.service_detail_id.loc[0]
# service_detail_id
df_service_mapping

,city,service_name,service_detail_id,city_id,service_id
0,Lucknow,Link,5c1248c10241913903a42fd6,5bc5ae9612477c2ece7695af,572e29b0116b5db3057bd821


In [6]:
## pricing.fare_estimates_enriched & Order_logs_snapshot


query = f""" 
            WITH city_cluster_hex AS (

                    SELECT
                        cch.hex_id AS hex_id,
                        cch.cluster AS cluster
                    FROM
                        datasets.city_cluster_hex cch
                    WHERE
                        cch.resolution = 8
                        AND cch.cluster != ''
                ),

                fare_estimates AS (

                    SELECT
                        fe_ench.yyyymmdd AS yyyymmdd,
                        fe_ench.quarter_hour AS quarter_hour,
                        CASE 
                        WHEN CAST(SUBSTR(fe_ench.quarter_hour, 1,2) AS INT) >= 8 AND CAST(SUBSTR(fe_ench.quarter_hour, 1,2) AS INT) <= 11 THEN '1.Morning Peak'
                        WHEN CAST(SUBSTR(fe_ench.quarter_hour, 1,2) AS INT) >= 17 AND CAST(SUBSTR(fe_ench.quarter_hour, 1,2) AS INT) <= 21 THEN '3.Evening Peak'
                        WHEN CAST(SUBSTR(fe_ench.quarter_hour, 1,2) AS INT) > 11 AND CAST(SUBSTR(fe_ench.quarter_hour, 1,2) AS INT) < 17 THEN '2.Afternoon'
                        ELSE '4.Rest' END AS time_period,
                        CAST(SUBSTR(fe_ench.quarter_hour, 1,2) AS INT) AS hour,
                        fe_ench.city AS city_name,
                        fe_ench.service_name AS service_name,
                        fe_ench.service_detail_id AS service_detail_id,
                        fe_ench.pickup_location_hex_8 AS pickup_location_hex_8,
                        fe_ench.fare_estimate_id AS fare_estimate_id,
                        fe_ench.user_id  AS customer_id

                    FROM
                        pricing.fare_estimates_enriched fe_ench
                    WHERE
                        fe_ench.yyyymmdd >= '{startdate}'
                        AND fe_ench.yyyymmdd <= '{enddate}'
                        AND fe_ench.service_detail_id IN
                                ('5c1248c10241913903a42fd6', '649564c80b573a42c10a4a44')                            
                ),

                orders AS (

                    SELECT
                        ols.yyyymmdd AS yyyymmdd,
                        ols.service_detail_id AS service_detail_id,
                        ols.customer_id AS customer_id,
                        ols.estimate_id AS fare_estimate_id,
                        ols.order_status AS order_status,
                        ols.order_id AS order_id,
                        ols.spd_fraud_flag AS spd_fraud_flag,
                        ols.discount AS discount,
                        ols.sub_total AS sub_total,
                        ols.rate_card_amount AS rate_card_amount,
                        ols.accept_to_pickup_distance  AS accept_to_pickup_distance,
                        ROW_NUMBER() OVER (PARTITION BY ols.order_id ORDER BY ols.updated_epoch DESC) AS row_number,
                        CASE 
                        WHEN ols.order_status IN ('dropped') AND ols.spd_fraud_flag != True THEN 'net_orders'
                        WHEN ols.order_status IN ('customerCancelled') AND ols.cancel_reason IN ('order cancelled before rider accepted') THEN 'cobra'
                        WHEN ols.order_status IN ('customerCancelled') AND ols.cancel_reason IN ('Order cancelled before rider was mapped') THEN 'cobrm'
                        WHEN ols.order_status IN ('customerCancelled') AND ols.cancel_reason NOT IN ('order cancelled before rider accepted', 'Order cancelled before rider was mapped') THEN 'ocara'
                        WHEN ols.order_status IN ('expired') AND length(ols.map_riders) < 28 THEN 'stockout'
                        WHEN ols.order_status IN ('expired') AND length(ols.map_riders) >= 28 THEN 'expiry_mapped' 
                        ELSE 'Other' 
                        END AS order_state

                    FROM
                        orders.order_logs_snapshot ols
                    WHERE
                        ols.yyyymmdd >= '{startdate}'
                        AND ols.yyyymmdd <= '{enddate}'
                        AND ols.service_detail_id IN 
                                    ('5c1248c10241913903a42fd6', '649564c80b573a42c10a4a44')
                )


                    SELECT 
                        fe.yyyymmdd AS yyyymmdd,
                        fe.city_name AS city_name,
                        fe.service_name AS service_name,
                        fe.service_detail_id AS service_detail_id,
                        pic.cluster AS pickup_location,
                        fe.pickup_location_hex_8 AS pickup_hex_8,
                        fe.customer_id AS fe_customer_id,
                        COUNT(DISTINCT fe.fare_estimate_id) AS fe_count,
                        COUNT(DISTINCT order_id) AS requested_orders,
                        COUNT(DISTINCT CASE WHEN order_status = 'dropped' AND spd_fraud_flag != true THEN order_id END) net_orders,

                        COUNT(DISTINCT CASE WHEN order_state IN ('cobra') AND row_number = 1 THEN order_id END) AS cobra,
                        COUNT(DISTINCT CASE WHEN order_state IN ('ocara') AND row_number = 1 THEN order_id END) AS ocara,
                        COUNT(DISTINCT CASE WHEN accept_to_pickup_distance > 0 THEN order_id END) AS accepted_orders,
                        COUNT(DISTINCT CASE WHEN order_state IN ('cobrm') AND row_number = 1 THEN order_id END) AS cobrm,
                        COUNT(DISTINCT CASE WHEN order_state IN ('stockout') AND row_number = 1 THEN order_id END) AS stockout,
                        COUNT(DISTINCT CASE WHEN order_state IN ('expiry_mapped') AND row_number = 1 THEN order_id END) AS expiry_mapped

                    FROM
                        fare_estimates fe
                    LEFT JOIN
                        city_cluster_hex pic
                        ON fe.pickup_location_hex_8 = pic.hex_id

                    LEFT JOIN
                        orders ord
                        ON fe.yyyymmdd = ord.yyyymmdd
                        AND fe.service_detail_id = ord.service_detail_id
                        AND fe.fare_estimate_id = ord.fare_estimate_id

                    GROUP BY 1,2,3,4,5,6,7

        """
df_temp = pd.read_sql(query, connection)

raw_fe_orders_data = df_temp

In [7]:
raw_fe_orders_data.head()

,yyyymmdd,city_name,service_name,service_detail_id,pickup_location,pickup_hex_8,fe_customer_id,fe_count,requested_orders,net_orders,cobra,ocara,accepted_orders,cobrm,stockout,expiry_mapped
0,20231004,Lucknow,Link,5c1248c10241913903a42fd6,Lucknow Zoo,883d8dcd51fffff,651cf3ef1b8c47f0a2677bb4,1,0,0,0,0,0,0,0,0
1,20231001,Lucknow,Link,5c1248c10241913903a42fd6,Kalyanpur west,883d8dce35fffff,642d70e04a48fb9e5ad434be,1,0,0,0,0,0,0,0,0
2,20231002,Lucknow,Link,5c1248c10241913903a42fd6,Telibagh,883d8d52e3fffff,64a71370d1af070b25803643,6,1,1,0,0,1,0,0,0
3,20231004,Lucknow,Link,5c1248c10241913903a42fd6,Telibagh,883d8d52e3fffff,645fb7880aeb4df2434a3186,2,1,1,0,0,1,0,0,0
4,20231002,Lucknow,Bike Lite,649564c80b573a42c10a4a44,Preeti Nagar,883d8dcf59fffff,61195c378902094dcf50d7ed,1,0,0,0,0,0,0,0,0


In [8]:
raw_fe_orders_data.to_csv('/Users/rapido/local-datasets/affluence/lucknow/raw_fe_orders_data_{}_{}_{}_{}.csv' \
                          .format(city, service,startdate, enddate)
                          , index = False)

In [ ]:
## datasets.iallocator_customer_segments

iallocator_customer_segments = []

for date in date_range:
    date_value = date.strftime('%Y%m%d')
    cs_query = f""" 
                SELECT 
                    DATE_FORMAT(cast(run_date as date), '%Y%m%d') yyyymmdd,
                    customer_id,
                    CASE WHEN taxi_income_segment = 'HIGH_INCOME' THEN customer_id END AS taxi_high_income,
                    CASE WHEN taxi_income_segment = 'MEDIUM_INCOME' THEN customer_id END AS taxi_medium_income,
                    CASE WHEN taxi_income_segment = 'LOW_INCOME' THEN customer_id END AS taxi_low_income,

                    CASE WHEN customer_service_segments = 'LINK_ONLY' THEN customer_id END AS link_only_service,
                    CASE WHEN customer_service_segments = 'AUTO_ONLY' THEN customer_id END AS auto_only_service,
                    CASE WHEN customer_service_segments = 'BOTH' THEN customer_id END AS both_service,

                    CASE WHEN ps_tag_link = 'PS' THEN customer_id END AS link_ps,
                    CASE WHEN ps_tag_link = 'NPS' THEN customer_id END AS link_nps,

                    CASE WHEN ps_tag_auto = 'PS' THEN customer_id END AS auto_ps,
                    CASE WHEN ps_tag_auto = 'NPS' THEN customer_id END AS auto_nps

                    -- CASE WHEN fe_intent_trend_type = 'Stable' THEN customer_id END AS fe_intent_stable,
                    -- CASE WHEN fe_intent_trend_type = 'Increasing' THEN customer_id END AS fe_intent_increasing,
                    -- CASE WHEN fe_intent_trend_type = 'Declining' THEN customer_id END AS fe_intent_declining

                FROM 
                    datasets.iallocator_customer_segments
                WHERE
                    DATE_FORMAT(cast(run_date as date), '%Y%m%d') = '{date_value}'
                    AND taxi_recency_segment != 'INACTIVE'
                    AND (taxi_lifetime_last_ride_city = '{city}' 
                        OR 
                        link_lifetime_last_ride_city = '{city}' 
                        OR 
                        link_lifetime_last_ride_city = '{city}')
            """
    df_temp = pd.read_sql(cs_query, connection)
    iallocator_customer_segments.append(df_temp)

# Concatenate all the results into a single DataFrame
raw_iallocator_customer_segments = pd.concat(iallocator_customer_segments)

In [ ]:
raw_iallocator_customer_segments.head()

In [ ]:
raw_iallocator_customer_segments.to_csv('/Users/rapido/local-datasets/affluence/lucknow/raw_iallocator_customer_segments_{}_{}_{}_{}.csv' \
                                        .format(city, service,startdate, enddate)
                                        , index = False)

In [ ]:
## Read back csv

raw_fe_orders_data = pd.read_csv('/Users/rapido/local-datasets/affluence/lucknow/raw_fe_orders_data_{}_{}_{}_{}.csv' \
                          .format(city, service,startdate, enddate))
raw_iallocator_customer_segments = pd.read_csv('/Users/rapido/local-datasets/affluence/lucknow/raw_iallocator_customer_segments_{}_{}_{}_{}.csv' \
                                        .format(city, service,startdate, enddate))

In [ ]:
df_fe_orders_data = raw_fe_orders_data.copy(deep=True)
print(df_fe_orders_data.shape)

df_affinity_data = raw_iallocator_customer_segments.copy(deep=True)
print(df_affinity_data.shape)

In [ ]:
df_fe_orders_data.head(2)

In [ ]:
df_affinity_data.head(2)

## Merge FE/Orders and Customer Segments 

In [ ]:
df_raw_merge = pd.merge(df_fe_orders_data,
                        df_affinity_data,
                        how = 'left',
                        left_on = ['yyyymmdd', 'fe_customer_id'],
                        right_on = ['yyyymmdd', 'customer_id']
                       )
df_raw_merge.shape

In [ ]:
df_raw_merge

In [ ]:
df_combained = df_raw_merge\
                    .groupby(['city_name', 'service_name', 'service_detail_id', 'pickup_location', 'pickup_hex_8']) \
                    .agg(
                        fe_cus_count = pd.NamedAgg('fe_customer_id', 'nunique'),
                        fe_count = pd.NamedAgg('fe_count', 'sum'),
                        requested_orders = pd.NamedAgg('requested_orders', 'sum'),
                        net_orders = pd.NamedAgg('net_orders', 'sum'),
                        cobra = pd.NamedAgg('cobra', 'sum'),
                        ocara = pd.NamedAgg('ocara', 'sum'),
                        accepted_orders = pd.NamedAgg('accepted_orders', 'sum'),
                        cobrm = pd.NamedAgg('cobrm', 'sum'),
                        stockout = pd.NamedAgg('stockout', 'sum'),
                        expiry_mapped = pd.NamedAgg('expiry_mapped', 'sum'),
                        taxi_high_income = pd.NamedAgg('taxi_high_income', 'nunique'),
                        taxi_medium_income = pd.NamedAgg('taxi_medium_income', 'nunique'),
                        taxi_low_income = pd.NamedAgg('taxi_low_income', 'nunique'),
                        link_only_service = pd.NamedAgg('link_only_service', 'nunique'),
                        auto_only_service = pd.NamedAgg('auto_only_service', 'nunique'),
                        both_service = pd.NamedAgg('both_service', 'nunique'),
                        link_ps = pd.NamedAgg('link_ps', 'nunique'),
                        link_nps = pd.NamedAgg('link_nps', 'nunique'),
                        auto_ps = pd.NamedAgg('auto_ps', 'nunique'),
                        auto_nps = pd.NamedAgg('auto_nps', 'nunique')
                        ).reset_index()
df_combained.shape

In [ ]:
df_combained

In [ ]:
df_combained.pickup_hex_8.nunique()

In [ ]:
df_combained.groupby(['service_name']).agg({'pickup_hex_8':'nunique'})

In [ ]:
df_combained.groupby(['service_name', 'pickup_hex_8']).agg({'fe_count':'sum'}).reset_index() \
.pivot(index ='pickup_hex_8' , columns ='service_name', values =['fe_count'])

## Analysing affluence_hi_tag - Less 


      - High Income median customer % 
      - Demand should be high at least more than Q3 level 
      

In [ ]:
df_combained['high_income_%'] = round(df_combained['taxi_high_income']*100/df_combained['fe_cus_count'])
df_combained['high_income_thrshld'] = 32 ## df_combained['high_income_%'].median()


df_combained['affluence_tag'] = np.where(
                                        df_combained['high_income_%'] <= df_combained['high_income_thrshld'] , 
                                        'Less', 
                                        'High')

df_combained.head(2)

In [ ]:
df_combained.fe_count.quantile([ 0.25, 0.50, 0.75, 0.80, 0.85, 0.90, 0.95])

In [ ]:
df_combained['demand_bucket'] = \
            np.where(df_combained['fe_count'] <= df_combained.fe_count.quantile(0.5), 'Lowest',
            np.where(df_combained['fe_count'] <= df_combained.fe_count.quantile(0.75), 'Low',
            np.where(df_combained['fe_count'] <= df_combained.fe_count.quantile(0.85), 'High','Highest')))

In [ ]:
df_combained.groupby(['affluence_tag', 'demand_bucket']).pickup_hex_8.nunique()